In [1]:
using GLPK
using JuMP

# Branch & Bound

Solve the the following MILP by applying B&B:

\begin{align*}
 \min & -5.5 x_1 - 2.1 x_2 \\
 \mathrm{s.t.} & -x_1 + x_2 \leq 2 \\
 & 8 x_1 + 2 x_2 \leq 17 \\
 & x_1, x_2 \geq 0 \\
 & x_1, x_2 \in \mathbb{Z}
\end{align*}

In [2]:
function initial_problem()
    m = Model(GLPK.Optimizer)

    @variable(m, x[1:2])

    @objective(m, Min, -5.5*m[:x][1] - 2.1*m[:x][2])

    @constraint(m, -m[:x][1] + m[:x][2] <= 2)
    @constraint(m, 8*m[:x][1] + 2*m[:x][2] <= 17)
    @constraint(m, m[:x] .>= 0)
    
    return m
end

initial_problem (generic function with 1 method)

In [3]:
function solve_and_print!(LP)
    optimize!(LP)
    println("Termination status: ", termination_status(LP))
    println("Objective value = ", objective_value(LP))
    println("Optimal x = ", value.(LP[:x]))
end

solve_and_print! (generic function with 1 method)

## Root node LP0

In [4]:
LP0 = initial_problem()

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

In [5]:
solve_and_print!(LP0)

Termination status: OPTIMAL
Objective value = -14.08
Optimal x = [1.3, 3.3]


What is the status of node LP0 ? Do we need to branch on it ?

## Branch on LP0

### First child: $x_1 \leq \mathrm{?}$ 

In [6]:
LP1 = initial_problem()
# Add branching constraints
@constraint(LP1, LP1[:x][1] <= 1)
LP1

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

In [7]:
solve_and_print!(LP1)

Termination status: OPTIMAL
Objective value = -11.8
Optimal x = [1.0, 3.0]


What is the status of node LP1 ? Do we need to branch on it ?

### Second child: $x_1 \geq \mathrm{?}$ 

In [8]:
LP2 = initial_problem()
# Add branching constraints
@constraint(LP2, LP2[:x][1] >= 2)
LP2

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

In [9]:
solve_and_print!(LP2)

Termination status: OPTIMAL
Objective value = -12.05
Optimal x = [2.0, 0.5]


What is the status of node LP2 ? Do we need to branch on it ?

## Branch on LP2

### First child: $x_2 \leq \mathrm{?}$

In [10]:
LP3 = initial_problem()
# Add branching constraints
@constraint(LP3, LP3[:x][1] >= 2)
@constraint(LP3, LP3[:x][2] <= 0)
LP3

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

In [11]:
solve_and_print!(LP3)

Termination status: OPTIMAL
Objective value = -11.6875
Optimal x = [2.125, 0.0]


### Second child: $x_2 \geq \mathrm{?}$

In [12]:
LP4 = initial_problem()
# Add branching constraints
@constraint(LP4, LP4[:x][1] >= 2)
@constraint(LP4, LP4[:x][2] >= 1)
LP4

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x

In [13]:
solve_and_print!(LP4)

Termination status: INFEASIBLE
Objective value = -12.05
Optimal x = [2.0, 0.5]


## Conclusion

### What is the optimal solution?

### Justify by drawing the B&B tree.